# 🚀 DE Zoomcamp - Module 1 : Docker & Terraform

**Objectif** : Maîtriser Docker (containerisation) et Terraform (Infrastructure as Code)

**Date** : Janvier 2026

---

## 📋 Sommaire

### Partie 1 : Docker
1. [Concepts fondamentaux](#1-docker---concepts-fondamentaux)
2. [Image vs Container](#2-image-vs-container)
3. [Commandes essentielles](#3-commandes-docker-essentielles)
4. [Flags importants (-it, -d, --rm)](#4-flags-importants)
5. [Volumes et persistance](#5-volumes-et-persistance-des-données)
6. [Networking Docker](#6-networking-docker)
7. [Dockerfile](#7-dockerfile)
8. [Docker Compose](#8-docker-compose)
9. [Commandes du cours (Postgres, pgAdmin, Ingest)](#9-commandes-pratiques-du-cours)

### Partie 2 : Terraform
10. [Concepts IaC](#10-terraform---infrastructure-as-code)
11. [Architecture et State](#11-architecture-terraform)
12. [Workflow (init, plan, apply, destroy)](#12-workflow-terraform)
13. [Fichiers du cours (main.tf, variables.tf)](#13-fichiers-terraform-du-cours)
14. [Best Practices](#14-best-practices-terraform)

---

# PARTIE 1 : DOCKER 🐳

---

## 1. Docker - Concepts Fondamentaux

### Définition

**Docker** = Plateforme de containerisation qui empaquète une application + ses dépendances dans un environnement isolé et portable.

### Docker vs VM

```
VM       : App → OS complet → Hypervisor → Hardware (lourd, GB)
Docker   : App → Docker Engine → OS Hôte → Hardware (léger, MB)
```

| Critère | Docker | VM |
|---------|--------|-----|
| **Taille** | 5-500 MB | 2-20 GB |
| **Démarrage** | < 1 seconde | 30-60 secondes |
| **Overhead** | ~2-5% | ~40-60% |
| **Isolation** | Processus (namespace) | Hardware (hypervisor) |

### Ce que Docker N'EST PAS

- ❌ **Pas une VM** : partage le kernel de l'hôte
- ❌ **Pas Kubernetes** : K8s = orchestration de containers sur clusters
- ❌ **Pas du config management** : pas Ansible/Chef (Docker = images immuables)
- ❌ **Pas magique pour la sécurité** : containers partagent le kernel

### Cas d'usage idéaux

✅ Microservices, CI/CD, environnements de dev, applications web stateless

❌ Applications GUI desktop, kernel modules, systèmes temps-réel

## 2. Image vs Container

**Concept clé** - la distinction la plus importante à comprendre :

### IMAGE = Template immuable (la classe)
- Stockée une seule fois sur disque
- Contient le système de fichiers et les instructions
- Commande : `docker images`

### CONTAINER = Instance de l'image (l'objet)
- Tu peux créer plusieurs containers de la même image
- Chaque container a son propre cycle de vie
- Commande : `docker ps -a`

```
IMAGE (postgres:18)  ──┬──> Container 1 (pgdatabase-dev)
                       ├──> Container 2 (pgdatabase-test)
                       └──> Container 3 (pgdatabase-prod)
```

**Règle** : Tu dois supprimer les containers AVANT de pouvoir supprimer leur image.

In [ ]:
# Test que Docker fonctionne
!docker run hello-world

In [ ]:
# Lister les images locales
!docker images

In [ ]:
# Lister les containers (actifs)
!docker ps

# Lister TOUS les containers (y compris arrêtés)
!docker ps -a

## 3. Commandes Docker Essentielles

### Gestion des containers

| Commande | Action |
|----------|--------|
| `docker ps` | Containers actifs |
| `docker ps -a` | Tous les containers |
| `docker stop <id>` | Arrêter un container |
| `docker start <id>` | Redémarrer un container |
| `docker rm <id>` | Supprimer un container |
| `docker logs <id>` | Voir les logs |
| `docker exec -it <id> bash` | Entrer dans un container actif |

### Gestion des images

| Commande | Action |
|----------|--------|
| `docker images` | Lister les images |
| `docker pull <image>` | Télécharger une image |
| `docker rmi <image>` | Supprimer une image |
| `docker build -t <name> .` | Construire une image |

### Nettoyage

```bash
docker container prune     # Supprimer containers arrêtés
docker image prune         # Supprimer images non utilisées
docker volume prune        # Supprimer volumes non utilisés
docker system prune -a     # Tout nettoyer (⚠️ agressif)
```

### Shortcuts utiles

```bash
docker stop $(docker ps -q)           # Arrêter TOUS les containers actifs
docker rm $(docker ps -aq)            # Supprimer TOUS les containers
docker rmi $(docker images -q)        # Supprimer TOUTES les images
```

## 4. Flags Importants

### `-it` : Mode interactif avec terminal

```bash
docker run -it ubuntu bash
```

- `-i` (interactive) : garde STDIN ouvert → tu peux taper
- `-t` (tty) : alloue un pseudo-terminal → affichage propre
- `-it` ensemble = shell interactif complet

### `-d` : Mode détaché (background)

```bash
docker run -d postgres:18
```

- Le container tourne en arrière-plan
- Tu récupères ton terminal
- Idéal pour les services (DB, web servers)

### `--rm` : Auto-suppression

```bash
docker run --rm hello-world
```

- Supprime automatiquement le container après exécution
- Évite l'accumulation de containers arrêtés

### `--name` : Nommer le container

```bash
docker run --name mon-postgres postgres:18
```

### `--entrypoint` : Override le point d'entrée

```bash
# Au lieu de lancer Python, lance bash
docker run -it --entrypoint=bash python:3.13-slim
```

**Use case** : Explorer le filesystem d'une image, debug

### `-e` : Variables d'environnement

```bash
docker run -e POSTGRES_USER=root -e POSTGRES_PASSWORD=secret postgres:18
```

### `-p` : Port mapping

```bash
docker run -p 5432:5432 postgres:18
#          host:container
```

### Tableau récapitulatif

| Situation | Flags | Exemple |
|-----------|-------|---------|
| Service background | `-d` | `docker run -d nginx` |
| Shell interactif | `-it` | `docker run -it ubuntu bash` |
| Container temporaire | `-it --rm` | `docker run -it --rm python bash` |
| Debug une image | `-it --entrypoint=bash` | `docker run -it --entrypoint=bash python:3.13` |

## 5. Volumes et Persistance des Données

**Problème** : Les données dans un container sont éphémères. Si tu supprimes le container, tout est perdu !

**Solution** : Les volumes.

### Syntaxe

```bash
docker run -v CHEMIN_LOCAL:CHEMIN_CONTAINER image
docker run -v NOM_VOLUME:CHEMIN_CONTAINER image
```

### 3 types de volumes

#### 1. Bind mount (dev) - Lie un dossier local
```bash
docker run -v $(pwd):/app python:3.13-slim
```
- Ton dossier local ↔ accessible dans le container
- Synchronisé en temps réel
- **Use case** : Dev, notebooks, scripts

#### 2. Volume Docker (prod) - Géré par Docker
```bash
docker volume create postgres-data
docker run -v postgres-data:/var/lib/postgresql/data postgres:18
```
- Survit à la suppression du container
- **Use case** : Bases de données, données importantes

#### 3. Volume anonyme (temporaire)
```bash
docker run -v /data ubuntu
```
- Docker crée un volume sans nom
- **Use case** : Cache, fichiers temporaires

### Options utiles

```bash
# Read-only
-v $(pwd)/config.yml:/app/config.yml:ro

# Workdir automatique
-v $(pwd):/app -w /app

# Multiples volumes
-v $(pwd)/code:/app -v $(pwd)/data:/data -v logs:/logs
```

### Commandes de gestion

```bash
docker volume ls                    # Lister
docker volume inspect mon-volume    # Détails
docker volume rm mon-volume         # Supprimer
docker volume prune                 # Nettoyer les non-utilisés
```

## 6. Networking Docker

### Pourquoi ?

Par défaut, chaque container est isolé. Pour que 2 containers communiquent (ex: ton app Python → Postgres), ils doivent être sur le **même réseau**.

### Créer un network

```bash
docker network create pg-network
```

### Connecter des containers au network

```bash
# Container 1 : Postgres
docker run -d --network=pg-network --name pgdatabase postgres:18

# Container 2 : pgAdmin
docker run -d --network=pg-network --name pgadmin dpage/pgadmin4
```

### Résolution DNS automatique

Sur le même network, les containers se trouvent **par leur nom** :

```python
# Dans ton script Python (container sur pg-network)
connection = psycopg2.connect(
    host="pgdatabase",  # ← Le nom du container, pas localhost !
    port=5432,
    database="ny_taxi"
)
```

### Commandes réseau

```bash
docker network ls                   # Lister les networks
docker network inspect pg-network   # Voir les containers connectés
docker network rm pg-network        # Supprimer
```

## 7. Dockerfile

### C'est quoi ?

Un **Dockerfile** = recette pour construire une image Docker personnalisée.

### Structure de base

```dockerfile
# Image de base
FROM python:3.13-slim

# Définir le répertoire de travail
WORKDIR /app

# Copier les fichiers de dépendances
COPY requirements.txt .

# Installer les dépendances
RUN pip install -r requirements.txt

# Copier le code source
COPY . .

# Commande par défaut
ENTRYPOINT ["python", "ingest_data.py"]
```

### Instructions principales

| Instruction | Description |
|-------------|-------------|
| `FROM` | Image de base |
| `WORKDIR` | Répertoire de travail |
| `COPY` | Copier fichiers locaux → image |
| `RUN` | Exécuter une commande (pendant le build) |
| `ENV` | Définir une variable d'environnement |
| `EXPOSE` | Documenter le port utilisé |
| `ENTRYPOINT` | Commande principale (fixe) |
| `CMD` | Arguments par défaut (overridable) |

### Build et run

```bash
# Construire l'image
docker build -t mon-app:v1 .

# Lancer un container
docker run mon-app:v1
```

### ENTRYPOINT vs CMD

```dockerfile
# ENTRYPOINT = commande principale (ne change pas)
ENTRYPOINT ["python", "script.py"]

# CMD = arguments par défaut (peut être overridden)
CMD ["--verbose"]

# Résultat : python script.py --verbose
# docker run mon-image --quiet → python script.py --quiet
```

## 8. Docker Compose

### C'est quoi ?

**Docker Compose** = Outil pour définir et lancer des applications multi-containers via un fichier YAML.

### Pourquoi ?

Au lieu de lancer manuellement :
```bash
docker network create ...
docker run postgres ...
docker run pgadmin ...
docker run mon-app ...
```

Tu fais simplement :
```bash
docker compose up
```

### Exemple docker-compose.yml

```yaml
services:
  pgdatabase:
    image: postgres:18
    environment:
      POSTGRES_USER: root
      POSTGRES_PASSWORD: root
      POSTGRES_DB: ny_taxi
    volumes:
      - ny_taxi_postgres_data:/var/lib/postgresql/data
    ports:
      - "5432:5432"

  pgadmin:
    image: dpage/pgadmin4
    environment:
      PGADMIN_DEFAULT_EMAIL: admin@admin.com
      PGADMIN_DEFAULT_PASSWORD: root
    volumes:
      - pgadmin_data:/var/lib/pgadmin
    ports:
      - "8085:80"

volumes:
  ny_taxi_postgres_data:
  pgadmin_data:
```

### Commandes

```bash
docker compose up           # Lancer tous les services
docker compose up -d        # En mode détaché
docker compose down         # Arrêter et supprimer
docker compose ps           # État des services
docker compose logs         # Voir les logs
docker compose build        # Rebuild les images
```

### Avantages

- ✅ Network créé automatiquement (tous les services peuvent se voir par leur nom)
- ✅ Un seul fichier = toute la stack
- ✅ Facile à versionner dans Git
- ✅ Reproductible

## 9. Commandes Pratiques du Cours

### 🐘 Container Postgres

```bash
docker run -it --rm \
  -e POSTGRES_USER="root" \
  -e POSTGRES_PASSWORD="root" \
  -e POSTGRES_DB="ny_taxi" \
  -v ny_taxi_postgres_data:/var/lib/postgresql/data \
  -p 5432:5432 \
  --network=pg-network \
  --name pgdatabase \
  postgres:18
```

**Décomposition :**
- `-e POSTGRES_*` : Variables d'environnement pour configurer Postgres
- `-v ny_taxi_postgres_data:...` : Volume nommé pour persister les données
- `-p 5432:5432` : Expose le port Postgres
- `--network=pg-network` : Connecte au réseau custom
- `--name pgdatabase` : Nom DNS pour les autres containers

---

### 📊 Container pgAdmin

```bash
docker run -it --rm \
  -e PGADMIN_DEFAULT_EMAIL="admin@admin.com" \
  -e PGADMIN_DEFAULT_PASSWORD="root" \
  -v pgadmin_data:/var/lib/pgadmin \
  -p 8085:80 \
  --network=pg-network \
  --name pgadmin \
  dpage/pgadmin4
```

**Accès** : http://localhost:8085

**Connexion à Postgres depuis pgAdmin** :
- Host : `pgdatabase` (le nom du container, pas localhost !)
- Port : `5432`
- Username : `root`
- Password : `root`

---

### 🚕 Container d'ingestion (taxi data)

```bash
docker run -it --rm \
  --network=pipeline_default \
  taxi_ingest:v001 \
    --pg-user=root \
    --pg-pass=root \
    --pg-host=pgdatabase \
    --pg-port=5432 \
    --pg-db=ny_taxi \
    --target-table=yellow_taxi_trips_2021_1 \
    --chunksize=100000
```

**Note** : `taxi_ingest:v001` est une image custom construite avec un Dockerfile.

---

# PARTIE 2 : TERRAFORM 🏗️

---

## 10. Terraform - Infrastructure as Code

### Définition

**Terraform** = Outil pour créer, modifier et détruire de l'infrastructure cloud via du CODE.

### Avant/Après

```
❌ AVANT (Console Web)
1. Ouvre console GCP
2. Clique "Create Bucket"
3. Remplis le formulaire
4. Répète pour chaque ressource...

Problèmes: Pas reproductible, pas versionnable, erreurs humaines

✅ AVEC TERRAFORM
resource "google_storage_bucket" "data" {
  name     = "mon-bucket"
  location = "US"
}

$ terraform apply

Avantages: Reproductible, versionnable (Git), automatisable (CI/CD)
```

### Ce que Terraform N'EST PAS

| Terraform | Ce qu'il faut utiliser |
|-----------|------------------------|
| ❌ Config système (installer logiciels) | Ansible, Chef, Puppet |
| ❌ Déploiement d'apps | Docker, Kubernetes, GitHub Actions |
| ❌ Monitoring | Datadog, Prometheus, CloudWatch |
| ❌ Orchestration data | Airflow, Prefect, Dagster |

**Exemple concret :**
- **Terraform crée** la VM sur GCP ✅
- **Ansible installe** PostgreSQL sur cette VM ✅

## 11. Architecture Terraform

### Les 3 composants

```
┌─────────────────────────────────────────┐
│           TERRAFORM CLI                  │
│   (terraform init, plan, apply)         │
└─────────────────┬───────────────────────┘
                  │
                  ▼
┌─────────────────────────────────────────┐
│            PROVIDERS                     │
│   (google, aws, azure, kubernetes)       │
│   = Plugins qui parlent aux APIs cloud  │
└─────────────────┬───────────────────────┘
                  │
                  ▼
┌─────────────────────────────────────────┐
│          CLOUD PLATFORMS                 │
│   (GCP, AWS, Azure)                      │
└─────────────────────────────────────────┘
```

### Le State File (`terraform.tfstate`)

**LE fichier le plus important !**

C'est un fichier JSON qui enregistre l'état actuel de ton infrastructure :

```json
{
  "resources": [
    {
      "type": "google_storage_bucket",
      "name": "demo-bucket",
      "attributes": {
        "id": "mon-projet-terra-bucket",
        "location": "US"
      }
    }
  ]
}
```

**Rôle :** Terraform compare ce fichier avec ton code `.tf` pour déterminer :
- Ce qui existe déjà
- Ce qui doit être créé
- Ce qui doit être modifié
- Ce qui doit être supprimé

⚠️ **Si tu perds ce fichier, Terraform ne sait plus ce qu'il a créé !**

## 12. Workflow Terraform

### Le flow standard

```
1. ÉCRIRE ──> main.tf, variables.tf
       │
       ▼
2. INIT ──> $ terraform init
       │     → Télécharge les providers
       ▼
3. PLAN ──> $ terraform plan
       │     → Preview des changements (rien n'est créé)
       ▼
4. APPLY ──> $ terraform apply
       │      → Crée/modifie les ressources
       ▼
5. DESTROY ──> $ terraform destroy
               → Supprime toutes les ressources
```

### Commandes essentielles

| Commande | Description | Quand |
|----------|-------------|-------|
| `terraform init` | Initialise le projet | 1ère fois + après ajout provider |
| `terraform plan` | Preview des changements | **Toujours avant apply** |
| `terraform apply` | Applique les changements | Créer/modifier l'infra |
| `terraform destroy` | Détruit tout | Nettoyage complet |
| `terraform fmt` | Formate le code | Avant commit Git |
| `terraform validate` | Vérifie la syntaxe | Debugging |
| `terraform show` | Affiche le state | Voir ce qui existe |

### Workflow sécurisé (production)

```bash
terraform plan -out=tfplan   # Sauvegarde le plan
# ← Vérifie le plan soigneusement
terraform apply tfplan       # Applique exactement ce plan
```

## 13. Fichiers Terraform du Cours

### Structure du projet

```
terrademo/
├── main.tf           # Ressources principales
├── variables.tf      # Définition des variables
├── terraform.tfstate # État (auto-généré, NE PAS COMMIT)
└── keys/
    └── my-CREDENTIALS.json  # Clé de service account GCP
```

---

### main.tf

```hcl
terraform {
  required_providers {
    google = {
      source  = "hashicorp/google"
      version = "7.16.0"
    }
  }
}

# Configuration du provider Google Cloud
provider "google" {
  credentials = file(var.credentials)
  project     = var.project
  region      = var.region
}

# Ressource : Google Cloud Storage Bucket (Data Lake)
resource "google_storage_bucket" "demo-bucket" {
  name          = var.gcs_bucket_name
  location      = var.location
  force_destroy = true

  lifecycle_rule {
    condition {
      age = 1  # Jours
    }
    action {
      type = "AbortIncompleteMultipartUpload"
    }
  }
}

# Ressource : BigQuery Dataset (Data Warehouse)
resource "google_bigquery_dataset" "demo_dataset" {
  dataset_id  = var.bq_dataset_name
  location    = var.location
  description = var.bq_dataset_description
}
```

---

### variables.tf

```hcl
variable "credentials" {
  description = "Path to GCP service account key"
  default     = "./keys/my-CREDENTIALS.json"
}

variable "project" {
  description = "GCP Project ID"
  default     = "cohesive-folio-485508-e4"
}

variable "region" {
  description = "GCP Region"
  default     = "us-central1"
}

variable "location" {
  description = "GCP Location for resources"
  default     = "US"
}

variable "gcs_bucket_name" {
  description = "GCS Bucket Name (must be globally unique)"
  default     = "cohesive-folio-485508-e4-terra-bucket"
}

variable "bq_dataset_name" {
  description = "BigQuery Dataset Name"
  default     = "demo_dataset"
}

variable "bq_dataset_description" {
  description = "BigQuery Dataset Description"
  default     = "Demo dataset created with Terraform"
}

variable "gcs_storage_class" {
  description = "Storage class for the bucket"
  default     = "STANDARD"
}
```

---

### Configuration des credentials

**Option 1 : Variable d'environnement**
```bash
export GOOGLE_CREDENTIALS='/chemin/vers/my-CREDENTIALS.json'
```

**Option 2 : Dans le code (comme ci-dessus)**
```hcl
credentials = file(var.credentials)
```

## 14. Best Practices Terraform

### 1. Ne JAMAIS commit le state dans Git

```bash
# .gitignore
terraform.tfstate
terraform.tfstate.backup
.terraform/
*.tfvars  # Contient souvent des secrets
```

**Solution pro :** Remote backend (GCS, S3)
```hcl
terraform {
  backend "gcs" {
    bucket = "mon-projet-terraform-state"
    prefix = "terraform/state"
  }
}
```

### 2. Utilise des variables (pas de valeurs en dur)

```hcl
# ❌ MAUVAIS
resource "google_storage_bucket" "data" {
  name = "cohesive-folio-485508-e4-terra-bucket"
}

# ✅ BON
resource "google_storage_bucket" "data" {
  name = "${var.project_id}-data-lake"
}
```

### 3. Nomme clairement tes ressources

```hcl
# ❌ MAUVAIS
resource "google_storage_bucket" "b1" { ... }

# ✅ BON
resource "google_storage_bucket" "raw_data_lake" { ... }
```

### 4. Toujours `plan` avant `apply`

```bash
terraform plan -out=tfplan
# Vérifie le plan
terraform apply tfplan
```

### 5. Utilise des labels

```hcl
resource "google_storage_bucket" "data" {
  name = "mon-bucket"
  
  labels = {
    environment = "dev"
    team        = "data-engineering"
    managed_by  = "terraform"
  }
}
```

**Utilité :** Facturation par équipe, filtrage, automatisation

### 6. Documente ton code

```hcl
# Ce bucket stocke les données brutes depuis les APIs
# Lifecycle: fichiers > 90 jours passent en NEARLINE
resource "google_storage_bucket" "raw_ingestion" {
  name          = "${var.project_id}-raw-data"
  storage_class = "STANDARD"
  
  lifecycle_rule {
    condition { age = 90 }
    action { 
      type          = "SetStorageClass"
      storage_class = "NEARLINE"
    }
  }
}
```

---

## 🎯 Récapitulatif Final

### Docker - Points clés

| Concept | Résumé |
|---------|--------|
| **Image vs Container** | Image = template, Container = instance |
| **Flags essentiels** | `-it` (interactif), `-d` (background), `--rm` (auto-clean) |
| **Volumes** | Bind mount (dev), Named volume (prod) |
| **Networking** | `--network` pour que les containers communiquent |
| **Docker Compose** | Un fichier YAML = toute la stack |

### Terraform - Points clés

| Concept | Résumé |
|---------|--------|
| **IaC** | Infrastructure définie en code, reproductible |
| **State** | `terraform.tfstate` = source de vérité |
| **Workflow** | init → plan → apply → destroy |
| **Variables** | Paramétrise ton code pour réutilisation |
| **Providers** | Plugins pour parler aux clouds (google, aws...) |

### Ta stack DE

```
Infrastructure (Terraform)
    ↓
Containerisation (Docker)
    ↓
Orchestration data (Airflow)
    ↓
Monitoring (Grafana/Datadog)
```

---

## 📚 Ressources

**Docker**
- [Documentation officielle](https://docs.docker.com/)
- [Docker Hub](https://hub.docker.com/)

**Terraform**
- [Terraform Docs](https://developer.hashicorp.com/terraform/docs)
- [Google Provider](https://registry.terraform.io/providers/hashicorp/google/latest/docs)
- [Learn Terraform](https://learn.hashicorp.com/terraform)

**Zoomcamp**
- [DE Zoomcamp GitHub](https://github.com/DataTalksClub/data-engineering-zoomcamp)